# Saved States Overview

## View turn pickles

In [ ]:
from aquawar import game

# save_dir = "saves/ai_battles/ai_battle_028"
save_dir = "saves/llama3.2_1b_single/llama3.2_1b_single/round_013"

ModuleNotFoundError: No module named 'core'

In [23]:
import os
pkl_files = []
for file in os.listdir(save_dir):
    if file.endswith(".pkl"):
        pkl_files.append(file)

print("Found pkl files:", pkl_files)

Found pkl files: ['turn_005.pkl', 'turn_000.pkl', 'latest.pkl', 'turn_006.pkl', 'turn_007.pkl', 'turn_003.pkl']


In [ ]:
import pickle
import json
import os
# select_files = pkl_files.copy()
# select_files = ["turn_002.pkl"]
save_dir = ""
select_files = ['good_save/latest.pkl']
select_files = ['saves/llama3_1_8b_majority/llama3.1_8b_single/round_001/v2_001.pkl']

for file in select_files:
    with open(os.path.join(save_dir, file), "rb") as f:
        state = pickle.load(f)
        print(f"=== Loaded state from {file}: ===")
        print(f"Keys:")
        for k, v in state.items():
            if type(v) is dict:
                print(f"  {k}: {[f'{kk}: {type(vv).__name__}' for kk, vv in v.items()]} | {len(v)} items")
            elif type(v) is list and len(v) > 0 and type(v[-1]) is dict:
                print(f"  {k}: list({[f'{kk}: {type(vv).__name__}' for kk, vv in v[-1].items()]}) | {len(v)} items")
        print(json.dumps(state, indent=2))


FileNotFoundError: [Errno 2] No such file or directory: 'saves/llama3.1_8b_single/gpt-oss_20b_single/round_006/v1_011.pkl'

## Voter pickles

In [8]:
from aquawar.persistent import PersistentGameManager
from aquawar.ai.ollama_majority import OllamaVoter
from aquawar.ai.ollama_player import OllamaPlayer
import pickle

# 1. Load a mid-game turn file
pm = PersistentGameManager('saves')
game = pm.load_game_state('llama3.1_8b_single', 'gpt-oss_20b_single', round_num=6, turn=10)

# 2. Set up a voter
voter = OllamaVoter(voter_index=1, name='Voter1', model='llama3.1:8b')
other_player = OllamaPlayer(name='OtherPlayer', model='gpt-oss:20b')
voter._other_player = other_player
voter.set_game_context(game, player_index=0)

# 3. Run a voter move (simulate 'action' phase)
history_entry, result, context = voter.make_voter_move(
    phase='action',
    save_dir='saves',
    round_num=6
)

# 4. Load and compare files
turn_path = 'saves/llama3.1_8b_single/gpt-oss_20b_single/round_006/turn_010.pkl'
voter_path = 'saves/llama3.1_8b_single/gpt-oss_20b_single/round_006/v1_011.pkl'

with open(turn_path, 'rb') as f:
    turn_data = pickle.load(f)
with open(voter_path, 'rb') as f:
    voter_data = pickle.load(f)

# Compare keys and structure
print('Turn file keys:', turn_data.keys())
print('Voter file keys:', voter_data.keys())
print('Game turn:', turn_data['state']['game_turn'], voter_data['state']['game_turn'])
print('History length:', len(turn_data['history']), len(voter_data['history']))
print('Evaluation:', turn_data['evaluation'], voter_data['evaluation'])

Turn file keys: dict_keys(['state', 'history', 'evaluation', 'players'])
Voter file keys: dict_keys(['state', 'history', 'evaluation', 'players'])
Game turn: 10 10
History length: 10 10
Evaluation: {'players': {'1': {'current_hp': 880, 'damage_dealt': 150, 'damage_taken': 800, 'assertions': {'true': 0, 'false': 2, 'skipped': 0}, 'invalid_moves': {'total': 0, 'by_type': {'invalid_response': 0, 'invalid_parameter': 0, 'invalid_action': 0}}}, '2': {'current_hp': 1250, 'damage_dealt': 320, 'damage_taken': 400, 'assertions': {'true': 1, 'false': 1, 'skipped': 0}, 'invalid_moves': {'total': 0, 'by_type': {'invalid_response': 0, 'invalid_parameter': 0, 'invalid_action': 0}}}}, 'game_status': 'ongoing'} {'players': {'1': {'current_hp': 880, 'damage_dealt': 270, 'damage_taken': 800, 'assertions': {'true': 0, 'false': 2, 'skipped': 0}, 'invalid_moves': {'total': 0, 'by_type': {'invalid_response': 0, 'invalid_parameter': 0, 'invalid_action': 0}}}, '2': {'current_hp': 1130, 'damage_dealt': 320, 'd

# Inference testing

In [13]:
from langchain_ollama import ChatOllama
model = "gpt-oss:20b"
temperature = 0.1
top_p = 0.9

llm = ChatOllama(
    model=model,
    temperature=temperature,
    top_p=top_p,
)
response = llm.invoke("What is the capital of France?")
print("LLM response:", response)

KeyboardInterrupt: 

In [40]:
import json
res_json = json.loads(response.model_dump_json())
print(json.dumps(res_json, indent=2))

{
  "content": "The capital of France is **Paris**.",
  "additional_kwargs": {},
  "response_metadata": {
    "model": "gpt-oss:20b",
    "created_at": "2025-08-15T18:18:15.029567396Z",
    "done": true,
    "done_reason": "stop",
    "total_duration": 90705936382,
    "load_duration": 89579361336,
    "prompt_eval_count": 74,
    "prompt_eval_duration": 577526484,
    "eval_count": 40,
    "eval_duration": 548376871,
    "model_name": "gpt-oss:20b"
  },
  "type": "ai",
  "name": null,
  "id": "run--f33e3fa7-0680-4afc-9776-3f3a5d116d0e-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 74,
    "output_tokens": 40,
    "total_tokens": 114
  }
}
